# Utils


In [ ]:
# | default_exp pdf.utils

In [ ]:
# | export

from dreamai.imports import *
from langchain_ray.imports import *
from langchain_ray.utils import *
from langchain_ray.chains import *

In [ ]:
for x in resolve_data_path("../././../resumes_5"):
    print(x)

../../resumes_5/0cf20170-8051-41ba-9060-1a82d43f4289.pdf
../../resumes_5/0d078ca0-b694-482e-a080-8bf9401a7d90.pdf
../../resumes_5/0f479ee8-5fd9-4f55-b254-5e8feef08038.pdf
../../resumes_5/0bedb223-262c-4388-9756-093dd7905428.pdf
../../resumes_5/0caeea25-ce38-4f7d-b4ac-c5ec9b7c4d84.pdf


In [ ]:
# | export


def pdf_files(pdf_folder):
    return get_files(pdf_folder, extensions=[".pdf"])


def pdf_docs(pdf, splitter, proc=True):
    docs = PyPDFLoader(str(pdf)).load_and_split(text_splitter=splitter)
    if proc:
        docs = [proc_doc_text(doc) for doc in docs]
    return docs


def pdf_to_docs(
    path,
    splitter=None,
    chunk_size=200,
    chunk_overlap=60,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    add_start_index=True,
    proc=True,
):
    if splitter is None:
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=separators,
            add_start_index=add_start_index,
        )
    docs = []
    for pdf in resolve_data_path(path):
        if Path(pdf).suffix == ".pdf":
            docs.append(pdf_docs(pdf, splitter=splitter, proc=proc))
    return docs


def text_cat(
    text,
    cats_model,
    thresh=0.6,
    classes=["Work Experience", "Education", "Certifications", "Other"],
    other_class="Other",
):
    if other_class not in classes:
        classes.append(other_class)
    ot_id = classes.index(other_class)
    probs, ids = cats_model.predict_proba(text).detach().cpu().max(1)
    ids[probs < thresh] = ot_id
    pred_classes = np.array(classes)[ids]
    return pred_classes.tolist()


def docs_cat(docs, cats_model):
    cats = text_cat([doc.page_content for doc in docs], cats_model)
    for doc, cat in zip(docs, cats):
        doc.metadata["category"] = cat
    return docs

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()